# 1. 데이터셋 살펴보기

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
data_dir = '../../data/DRIVING-DATASET/Detection/'
data_df = pd.read_csv(os.path.join(data_dir, 'df.csv'))
data_df.head()

In [ ]:
# 이미지 한개 살펴보기
index = 859
img_files = [fn for fn in os.listdir(os.path.join(data_dir,'images')) if fn.endswith('jpg')] #이미지들 리스트
img_file = img_files[index]
img_path = os.path.join(data_dir,'images' ,img_file)

In [ ]:
img_path

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# 바운딩 박스 그리기

BOX_COLOR = {'Bus':(200, 0, 0), 'Truck':(0, 0, 200)}
CLASS_ID_TO_NAME = {0: 'Bus', 1: 'Truck'}

def visualize(image, bounding_boxes,class_ids,color=BOX_COLOR, thickness=2):
    
    for class_id, bounding_box in zip(class_ids,bounding_boxes): 
        class_name = CLASS_ID_TO_NAME[class_id]
        x_min, y_min, x_max, y_max = map(int,bounding_box) #정수의 값을 넣어야 그려짐

        
    
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color[class_name], thickness=thickness)
        
        ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
        cv2.rectangle(image, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), color[class_name], -1)
        cv2.putText(
            image,
            text=class_name,
            org=(x_min, y_min - int(0.3 * text_height)),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=0.35, 
            color=(255,255,255), 
            lineType=cv2.LINE_AA,
        )
    return image

In [ ]:
CLASS_NAME_TO_ID = {'Bus': 0, 'Truck': 1} # 버스, 트럭의 키값을 가진 딕셔너리 -> string의 classid를 수치화하기위함

In [ ]:
# 이미지 그리기 위한 데이터
img_id = img_file.split('.')[0]
img_df = data_df[data_df['ImageID']==img_id]
class_names = img_df['LabelName'].values
class_ids = np.array([CLASS_NAME_TO_ID[class_name] for class_name in class_names])
bounding_box = img_df[["XMin", "XMax", "YMin", "YMax"]].values
bounding_box[:, [1,2]] = bounding_box[:, [2,1]]

In [ ]:
class_names

In [ ]:
class_ids

In [ ]:
# 이미지 크기에 따라서 노멀라이즈 되어있음, 이미지 크기를 곱해서 반환해서 사용해야함
bounding_box

In [ ]:
img_h, img_w, _ = img.shape

class_id = CLASS_NAME_TO_ID[class_names[0]]

# 노멀라이즈 되어있는 바운딩 박스의 좌표를 복원

bounding_box[:,[0,2]] *= img_w
bounding_box[:,[1,3]] *= img_h

In [ ]:
bounding_box

In [ ]:
canvas = visualize(img, bounding_box, class_ids)
plt.figure(figsize=(6,6))
plt.imshow(canvas)
plt.show()

In [ ]:
# 인덱스별 이미지 확인
from ipywidgets import interact
img_files = [fn for fn in os.listdir(os.path.join(data_dir,'images')) if fn.endswith('jpg')] #이미지들 리스트

In [ ]:
@interact(index=(0, len(img_files)-1))
def show_imgbox(index=1):
    img_file = img_files[index]
    img_path = os.path.join(data_dir, 'images',img_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    

    img_id = img_file.split('.')[0]
    img_df = data_df[data_df['ImageID']==img_id]
    bounding_box = img_df[['XMin','XMax','YMin','YMax']].values
    bounding_box[:, [1,2]] = bounding_box[:, [2,1]]

    img_h, img_w, _ = img.shape
    bounding_box[:,[0,2]] *= img_w
    bounding_box[:,[1,3]] *= img_h   
    
    class_name = img_df['LabelName'].values
    class_ids = np.array([CLASS_NAME_TO_ID[class_name] for class_name in class_names])

    canvas = visualize(img, bounding_box, class_ids)
    plt.figure(figsize=(6,6))
    plt.imshow(canvas)
    plt.show()

# 2. 데이터 셋 및 데이터로더 구축

In [ ]:
import torch
import numpy as np

In [ ]:
class My_Dataset():
    def __init__(self, data_dir, phase, transformer=None):
        self.data_dir = data_dir
        self.phase = phase
        self.transformer = transformer

        self.data_df = pd.read_csv(os.path.join(self.data_dir, 'df.csv'))
        self.img_files = [fn for fn in os.listdir(os.path.join(self.data_dir,phase)) if fn.endswith('jpg')]

        self.CLASS_NAME_TO_ID = {'Bus': 0, 'Truck': 1}
        
    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, index):
        #input = img
        #target = label(box, class_id)
        #get_image로 filename을 가지고오고 이것을 이용해 get_label로 box와 classid를 가져옴

        img, filename = self.get_img(index)
        boxes, class_ids = self.get_label(filename)

        img_h, img_w,_ = img.shape

        if self.transformer :
            img = self.transformer(img)
            _, img_h, img_w = img.shape #transformer에서 imgresize가 되었을수도있음

        boxes[:,[0,2]] *= img_w
        boxes[:,[1,3]] *= img_h

        target = {}
        target['boxes'] = torch.Tensor(boxes).float() #regression을 수행할것이라 float
        target["labels"] = torch.Tensor(class_ids).long() #int형

        return img, target, filename
        

    #getitem의 기능을 보충하기 위해서 생성
    def get_img(self, index):
        filename = self.img_files[index]

        img_path = os.path.join(self.data_dir,self.phase,filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        return img, filename

    def get_label(self, filename):
        img_id = filename.split('.')[0]
        img_df = self.data_df[self.data_df['ImageID']==img_id]
        boxes = img_df[['XMin','XMax','YMin','YMax']].values
        boxes[:, [1,2]] = boxes[:, [2,1]] # model [x_min, y_min, x_max, y_max] 포멧으로 받음

        class_names = img_df['LabelName'].values
        class_ids = np.array([self.CLASS_NAME_TO_ID[class_name] for class_name in class_names])

        return boxes,class_ids
        
        

In [ ]:
data_dir = '../../data/DRIVING-DATASET/Detection/'
dataset = My_Dataset(data_dir, 'train')

dataset[0]

In [ ]:
index = 5
image, target, filename = dataset[index]
boxes = target['boxes'].numpy()
class_ids = target['labels'].numpy()

canvas = visualize(image, boxes, class_ids)

plt.figure(figsize=(6,6))
plt.imshow(canvas)
plt.show()

In [ ]:
from torchvision import transforms

In [ ]:
IMG_SIZE = 448

transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size = (IMG_SIZE,IMG_SIZE)),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
def collate_fn(batch):
    img_list = []
    target_list = []
    filename_list = []

    for a,b,c in batch:
        img_list.append(a)
        target_list.append(b)
        filename_list.append(c)

    return img_list, target_list, filename_list

In [ ]:
data_dir = '../../data/DRIVING-DATASET/Detection/'
BATCH_SIZE = 6

trainset = My_Dataset(data_dir=data_dir, phase="train", transformer=transformer)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [ ]:
for index, batch in enumerate(trainloader):
    images = batch[0]
    targets = batch[1]
    filenames = batch[2]

    print(len(images),len(targets), len(filenames))
    
    if index == 0:
        break

    

In [ ]:
def My_DataLoader(data_dir,transfomer, batch_size=4):

    data_loaders = {}

    train_dataset = My_Dataset(data_dir, 'train', transformer)
    data_loaders['train'] = DataLoader(train_dataset, batch_size = batch_size, shuffle=True, collate_fn = collate_fn)

    val_dataset = My_Dataset(data_dir, 'val', transformer)
    data_loaders['val'] = DataLoader(val_dataset, batch_size = 1, shuffle=False, collate_fn = collate_fn)

    return data_loaders

In [ ]:
data_dir = '../../data/DRIVING-DATASET/Detection/'
dloaders = My_DataLoader(data_dir,transformer,batch_size=4)

for phase in ["train", "val"]:
    for index, batch in enumerate(dloaders[phase]):
        img = batch[0]
        targets = batch[1]
        filenames = batch[2]
        print(len(img),len(targets), len(filenames))
        
        if index == 0:
            break

# 3. Faster RCNN 모델 불러와서 사용하기

In [ ]:
from torchvision import models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor #모델 변경할 떄 사용

In [ ]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True) #ResNet-50 아키텍처와 Feature Pyramid Network기반
model
#box_predictor 의 cls_score - class의 수, bbox_pred - class수 *4(사각형 꼭지점)

In [ ]:
in_features = model.roi_heads.box_predictor.cls_score.in_features # 기존의 input값
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2) #2 : 2개의 클래스로 변경

In [ ]:
#변경 확인
model

In [ ]:
def build_model(num_class):
    model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_class)

    return model

In [ ]:
model = build_model(2)

# 4. 학습코드 작성

In [ ]:
#모델이 train모드일 때 list형태의 img와 target을 받으면 loss를 반환해줌(FasterRCNN내부에 손실 계산 기능이 있음)
 #eval모드일 때에는 image만 input으로 받음

In [ ]:
phase = 'train'
model.train()

for index, batch in enumerate(dloaders[phase]):
    imgs = batch[0]
    targets = batch[1]
    filenames = batch[2]

    # 이미지, 타겟을 리스트로 (굳이x)
    imgs = list(img for img in imgs)
    targets = [{k: v for k, v in t.items()} for t in targets]

    loss = model(imgs, targets)

    if index == 0:
        break

    

In [ ]:
#첫번째 스테이지단계에서 수행하는 CNN에 의한 loss
#loss_objectness : 어떤 대상에 대해서 분류를 하지 않고, regionproposal구간에 오브젝트가 있는지 없는지를 구분하는 loss
#loss_rpn_box_reg : Region Proposal Network에서 발견된 객체 후보들의 경계 상자regression box의 예측값과 Target 간의 차이를 측정
loss

In [ ]:
# model이 train모드일때는 이미지와 타겟을 받아서 loss를 계산해줌, eval모드일 때는 이미지만 받아서 예측값을 반환해줌
# 아래 train_one_epoch에서는 모델을 val부분에서까지 train모드로 사용 - loss 계산을 위함
# 하지만 with torch.set_grad_enabled(phase == "train"): 부분과 if phase == "train":에서만 backward와 grad를 계산했기 때문에 train모드로 val을 진행해도
# grad가 갱신되지 않아 학습에 영향을 미치지 않음

In [ ]:
from collections import defaultdict #매 에폭마다 loss를 담기 위해서 선언(존재하지 않는 키(key)에 접근하려고 할 때 에러가 발생하지 않고, 기본값을 반환)

In [ ]:
def train_one_epoch(dataloaders, model, optimizer, device):#모델이 직접 loss를 구해서 criterion이 필요없음

    train_loss = defaultdict(float)
    val_loss = defaultdict(float)

    model.train()
    model.to(device)

    for phase in ['train', 'val']:
        for index, batch in enumerate(dataloaders[phase]):
            imgs = batch[0]
            targets = batch[1]
            filename = batch[2]

            imgs = [img.to(device) for img in imgs]
            targets = [{k:v.to(device) for k, v in t.items()} for t in targets]

            with torch.set_grad_enabled(phase=='train'): #train일 경우에만 loss로 grad를 구함
                losses = model(imgs, targets)
                total_loss = sum(loss for loss in losses.values())

                if phase == 'train' :
                    optimizer.zero_grad()
                    total_loss.backward()
                    optimizer.step()

                    if (index>0) and (index%VERBOSE_FREQ == 0) : 
                        text = f'{index}/{len(dataloaders[phase])} -'
                        for k,v in losses.items():
                            text +=f'{k} : {v.item():.4f}'
                        print(text)

                        for k,v in losses.items():
                            train_loss[k] += v.item()
                        train_loss['total_loss'] += total_loss.item() #backward되는 부분이 loss에 포함되어있어서 item을 붙여줘야함

                else : 
                    for k, v in losses.items():
                        val_loss[k] += v.item()
                    val_loss['total_loss'] += total_loss.item()

    for k in train_loss.key():
        train_loss[k] /= len(dataloaders['train'])
        val_loss[k] /= len(dataloaders['val'])

    return train_loss, val_loss         


In [ ]:
data_dir = '../../data/DRIVING-DATASET/Detection/'
is_cuda = True

NUM_CLASSES = 2
IMAGE_SIZE = 448
BATCH_SIZE = 6
VERBOSE_FREQ = 30
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataloaders = My_DataLoader(data_dir,transformer,batch_size=4)
model = build_model(NUM_CLASSES)
model = model.to(DEVICE)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def save_model(model_state, model_name, save_dir='./trained_model'):
    os.mkdir(save_dir, exist_ok=True)
    torch.save(model_state, os.path.join(save_dir,model_name))

In [ ]:
num_epochs = 30

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_loss, val_loss = train_one_epoch(dataloaders, model, optimizer, DEVICE)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f"epoch:{epoch+1}/{num_epochs} - Train Loss: {train_loss['total_loss']:.4f}, Val Loss: {val_loss['total_loss']:.4f}")
    
    if (epoch+1) % 10 == 0:
        save_model(model.state_dict(), f'model_{epoch+1}.pth')

In [ ]:
tr_loss_classifier = [] 
tr_loss_box_reg = []
tr_loss_objectness = []
tr_loss_rpn_box_reg = []
tr_loss_total = []

for tr_loss in train_losses:
    tr_loss_classifier.append(tr_loss['loss_classifier'])
    tr_loss_box_reg.append(tr_loss['loss_box_reg'])
    tr_loss_objectness.append(tr_loss['loss_objectness'])
    tr_loss_rpn_box_reg.append(tr_loss['loss_rpn_box_reg'])
    tr_loss_total.append(tr_loss['total_loss'])

val_loss_classifier = [] 
val_loss_box_reg = []
val_loss_objectness = []
val_loss_rpn_box_reg = []
val_loss_total = []

for vl_loss in val_losses:
    val_loss_classifier.append(vl_loss['loss_classifier'])
    val_loss_box_reg.append(vl_loss['loss_box_reg'])
    val_loss_objectness.append(vl_loss['loss_objectness'])
    val_loss_rpn_box_reg.append(vl_loss['loss_rpn_box_reg'])
    val_loss_total.append(vl_loss['total_loss'])

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(tr_loss_total, label="train_total_loss")
plt.plot(tr_loss_classifier, label="train_loss_classifier")
plt.plot(tr_loss_box_reg,  label="train_loss_box_reg")
plt.plot(tr_loss_objectness, label="train_loss_objectness")
plt.plot(tr_loss_rpn_box_reg,  label="train_loss_rpn_box_reg")

plt.plot(val_loss_total, label="train_total_loss")
plt.plot(val_loss_classifier, label="val_loss_classifier")
plt.plot(val_loss_box_reg,  label="val_loss_box_reg")
plt.plot(val_loss_objectness, label="val_loss_objectness")
plt.plot(val_loss_rpn_box_reg,  label="val_loss_rpn_box_reg")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.grid("on")
plt.legend(loc='upper right')
plt.tight_layout()

# 5. Confidence threshold 와 Non-maximum suppression(NMS)적용

In [ ]:
#모델 불러오는 함수
def load_model(ckpt_path, num_class, device):
    
    checkpoint = torch.load(ckpt_path, map_location=device)
    model = build_model(num_class=num_class)
    model.load_state_dict(checkpoint)
    model = model.to(device)
    model.eval()

    return model

In [ ]:
# 학습된 모델 불러오기
NUM_CLASSES = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data_dir = '../../data/DRIVING-DATASET/Detection/'

model = load_model(ckpt_path='./trained_model/model_30.pth', num_class=NUM_CLASSES,device=DEVICE)

In [ ]:
model

In [ ]:
# 1개의 불러온 모델의 예측값에 대해 확인해보기 
for index, batch in enumerate(dataloaders['val']):
    imgs = batch[0]
    imgs = list(img.to(DEVICE) for img in imgs)

    with torch.no_grad():
        prediction = model(imgs)

    if index==0:
        break

In [ ]:
prediction

In [ ]:
from torchvision.ops import nms

In [ ]:
# 후처리함수
def postprocess(prediction, conf_threshold=0.2, IoU_threshold=0.1):
    pred_box = prediction['boxes'].cpu().detach() #detach() : Tensor의 그라디언트 계산을 멈추는 역할
    pred_label = prediction['labels'].cpu().detach()
    pred_score = prediction['scores'].cpu().detach()

    #conf_threchold
    valid_index_conf = pred_score>conf_threshold
    pred_box = pred_box[valid_index_conf]
    pred_label = pred_label[valid_index_conf]
    pred_score = pred_score[valid_index_conf]

    #nms_threshold
    #nms : box, score, IoU_threshold를 받음,
    #box좌표 : x1, y1, x2, y2로 들어감(FasterRCNN은 xmin,ymin, xmax,ymax의 순으로 박스를 반환해줘서 그대로 사용 가능)
    valid_index_nms = nms(pred_box, pred_score, IoU_threshold)
    pred_box = pred_box[valid_index_nms].numpy() # 이미지 그리기 위해서 numpy로 변환
    pred_label = pred_label[valid_index_nms].numpy()
    pred_score = pred_score[valid_index_nms].numpy()


    ##np.newaxis : 배열의 차원을 하나 늘려주는 역할 -> pred_box에 맞춰주기 위해서 나머지를 한차원씩 늘려줌
    return np.concatenate((pred_box,pred_score[:, np.newaxis], pred_label[:, np.newaxis]), axis=1)

    
    
    

In [ ]:
#np.newaxis 예시
        # # 1차원 배열
        # arr_1d = np.array([1, 2, 3])
        
        # # 1차원 배열을 2차원으로 확장
        # arr_2d = arr_1d[:, np.newaxis]
        
        # print(arr_2d.shape)  # 출력: (3, 1)
        
        # # 2차원 배열
        # arr_2d = np.array([[1, 2, 3]])
        
        # # 2차원 배열을 3차원으로 확장
        # arr_3d = arr_2d[:, :, np.newaxis]
        
        # print(arr_3d.shape)  # 출력: (1, 3, 1)

In [ ]:
# predictrion 확인
for index, batch in enumerate(dataloaders['val']):
    imges = batch[0]
    img = [img.to(DEVICE)for img in imges]

    with torch.no_grad():
        prediction = model(img)

    prediction = postprocess(prediction[0])

    if index==0:
        break


In [ ]:
prediction # box, score, label의 순서로 

In [ ]:
from torchvision.utils import make_grid

In [ ]:
# 예측하기
pred_imgs = []
pred_labels = []

for index, batch in enumerate(dataloaders['val']):
    imgs = batch[0]
    img = [img.to(DEVICE)for img in imgs]

    with torch.no_grad():
        prediction = model(img)

    prediction = postprocess(prediction[0])

    # 예측된 상자의 크기가 이미지의 크기보다 큰것을 방지
    # clip : 배열의 요소가 최솟값보다 작으면 최솟값으로, 최댓값보다 크면 최댓값으로 설정
    # xmax와 ymax를 제한
    prediction[:,2].clip(min = 0, max = imgs[0].shape[1])
    prediction[:,3].clip(min = 0, max = imgs[0].shape[0])

    pred_imgs.append(imgs[0])
    pred_labels.append(prediction)

    if index==30:
        break

In [ ]:
# 예측값 이미지로 살펴보기
@interact(index=(0, len(pred_imgs)-1))
def show_predict(index=0):
    int_labels = pred_labels[index][:, 5].astype(int)

    # 기존의 예측 이미지가 normalization되어있어서 색감이 이상함, 
    # make_grid의 normalize를 이용하면 pred_imgs[index]의 내부의 값을 0~1사이의 값으로 재조정
    # 이에 255를 곱해줘서 원래의 색감으로 돌아옴
    image = make_grid(pred_imgs[index], normalize=True).permute(1,2,0).numpy() 
    image = (image * 255).astype(np.uint8) #imshow가능하도록
    img = image.copy()
    
    result = visualize(img, pred_labels[index][:, 0:4], int_labels)
    
    plt.figure(figsize=(6,6))
    plt.imshow(result)
    plt.show()

# 6. 성능 검증을 위한 지표 

In [ ]:
import json
from pycocotools.coco import COCO # COCO 데이터셋을 로드하고 사용하기 위한 도구,데이터셋의 이미지, annotation, 클래스 레이블 등을 로드하고 검색
from pycocotools.cocoeval import COCOeval #모델의 성능을 평가하기 위한 도구, 모델의 예측 결과를 COCO 데이터셋의 실제 annotation과 비교(정확도, 재현율, mAP 등)

In [ ]:
annfile = '../../data/DRIVING-DATASET/Detection/val.json'#groundtruth 가 담겨있는 json파일

with open(annfile, 'r') as f:
    json_data = json.load(f)

imgToid = json_data['imageToid'] #filename을 이용해서 file의 image_id를 가져오고, 그 id를 이용해 annotation을 불러오기 위함

In [ ]:
# coco를 이용해 모델을 평가하기 위해서 coco에 annotation정보를 입력해 GT를 생성
cocoGT = COCO(annfile)

In [ ]:
#원래는 xmin,ymin,xmax,ymax의 포멧, coco는 xmin,ymin,w,h포멧이기때문에 바꿔줘야함
def changeformat(box):
    Xmin = box[:,0]
    Ymin = box[:,1]
    w = box[:,2] - box[:,0]
    h = box[:,3] - box[:,1]

    return np.stack((Xmin, Ymin, w, h), axis =1)

In [ ]:
# 모델 예측 구하기
COCO_anno = []

for index, batch in enumerate(dataloaders['val']):
    imgs = batch[0]
    filename = batch[2][0]
    
    img = list(img.to(DEVICE) for img in imgs)

    with torch.no_grad():
        prediction = model(img)

    prediction = postprocess(prediction[0])
    prediction[:,2].clip(min=0, max=img[0].shape[1])
    prediction[:,3].clip(min=0, max=img[0].shape[0])

    #박스포멧 바꿔주기
    box_xywh = changeformat(prediction[:, 0:4])
    score = prediction[:,4][:,np.newaxis] #[:,np.newaxis]:box포멧에 맞추기 위해
    cls_id = prediction[:,5][:,np.newaxis]
    img_id = np.array([imgToid[filename]]*len(cls_id))[:,np.newaxis] #* len(cls_id): 하나의 파일에 여러개의 오브젝트가 있을것이기 때문에 곱해서 개수를 맞춰줌

    COCO_anno.append(np.concatenate((img_id, box_xywh, score, cls_id), axis=1))

    if index %50 ==0:
        print(f"{index}/{len(dataloaders['val'])} Done")

COCO_anno = np.concatenate(COCO_anno, axis = 0) # 하나의 배열로 합쳐줌

In [ ]:
cocoDT = cocoGT.loadRes(COCO_anno) #모델이 예측한 COCO_anno데이터를 cocoGT와 같은 형식으로 변환

In [ ]:
annType = "bbox" #annotation중 어떤걸 평가할지 
cocoEval = COCOeval(cocoGT,cocoDT,annType) #COCOeval객체 생성
cocoEval.evaluate() #IoU를 계산하고, 평가를 수행하기 위한 준비
cocoEval.accumulate() #precision, recall, AP 등을 계산
cocoEval.summarize() #평가 결과를 요약하여 출력
eval_stats = cocoEval.stats # 평가 결과를 변수에 저장

In [ ]:
#cuda 내의 메모리 해제
torch.cuda.empty_cache()

# 7. 동영상 예측 

In [ ]:
from time import time

In [ ]:
video_path = '../../data/DRIVING-DATASET/sample_video.mp4'

In [ ]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(size=(IMG_SIZE,IMG_SIZE)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
@torch.no_grad()
def model_predict(img, model):
    img = transformer(img)
    img = img.to(DEVICE)

    prediction = model([img])

    return prediction

In [ ]:
vid = cv2.VideoCapture(video_path)

while(vid.isOpened):
    ret, frame = vid.read()

    if ret:
        since = time()
        original_h, original_w,_ = frame.shape
    
        prediction = model_predict(frame, model)
        prediction = postprocess(prediction[0])
        prediction[:,[0,2]] *= original_w/IMG_SIZE #박스의 크기를 프레임의 크기에 맞춰서 변환
        prediction[:,[1,3]] *= original_h/IMG_SIZE

        prediction[:,2].clip(min=0, max=original_w)
        prediction[:,3].clip(min=0, max=original_h)

        canvas = visualize(frame, prediction[:,0:4], prediction[:,5])
        text = f'{(time()-since)*1000:.0f}ms/image'
        cv2.putText(canvas, text, (20, 40), cv2.FONT_HERSHEY_PLAIN, 2, (255, 255, 255), 2)
        cv2.imshow('camera', canvas)

        key = cv2.waitKey(1)
        if key == 27:
            break
        if key == ord('s'):
            cv2.waitKey()
    
vid.release()
cv2.destroyAllWindows()